## Model Training

Let's train several models and tune their hyperparameters

In [1]:
import os
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass
class ModelTrainerConfig:
    """
    
    Configuration class for model training operations.

    This class contains all the parameters and paths needed for training
    machine learning models.

    """
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    cross_validation: int
    scoring: str
    available_models: List[str]
    target_column: str


In [ ]:
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml, create_directories
from src.datascience import logger
from src.datascience.config.configuration import DataIngestionConfig

class ConfigurationManager:
    """
    Configuration manager for handling YAML configuration files.
    
    This class loads configuration, parameters, and schema files and provides
    methods to retrieve specific configuration objects.
    """
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH ):

        """
        Initialize the ConfigurationManager.
        
        Args:
            config_filepath (Path): Path to the main configuration file
            params_filepath (Path): Path to the parameters file
            schema_filepath (Path): Path to the schema file
        """
          
        try:
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
            
            # Create artifacts root directory
            create_directories([self.config.artifacts_root])
            logger.info("ConfigurationManager initialized successfully")
            
        except Exception as e:
            logger.error(f"Error initializing ConfigurationManager: {e}")
            raise
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.models

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = config.target_column,
            cross_validation = int(config.cross_validation),
            scoring= config.scoring,
            available_models = config.available_models,
            target_column = config.target_column,   

            models = params
        )
        return model_trainer_config

[2025-08-02 19:31:49,796: INFO: __init__: Logger initialized for the datascience package.]


In [ ]:
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib
from typing import List

class ModelTrainer:
    """
    Component for training machine learning models.
    """
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.estimators = {}


    def _make_estimators(self):
        for name in self.config.available_models:
            if name == "logistic_regression":
                self.estimators[name] = LogisticRegression(max_iter=2000)
            elif name == "random_forest":
                self.estimators[name] = RandomForestClassifier(random_state=42, n_jobs=-1)
            elif name == "svm":
                # probability=True so we can use predict_proba for ROC AUC
                self.estimators[name] = SVC(probability=True)
        

    def train(self):
        """
        Trains several models and saves them.
        
        Raises:
            FileNotFoundError: If training or testing data files don't exist
            Exception: If there's an error during training
        """
        try:

            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)

            train_x = train_data.drop([self.config.target_column], axis=1)
            test_x  = test_data.drop([self.config.target_column], axis=1)
            train_y = train_data[self.config.target_column]
            test_y = test_data[self.config.target_column]

            

            # lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
            # lr.fit(train_x, train_y)
            # logger.info("Model training completed")

            # joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        except FileNotFoundError as e:
            logger.error(f"Data file not found: {e}")
            raise
        except Exception as e:
            logger.error(f"Error during model training: {e}")
            raise
        

In [5]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e 

[2025-08-02 19:31:56,230: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-08-02 19:31:56,231: INFO: common: YAML file: params.yaml loaded successfully]
[2025-08-02 19:31:56,233: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-08-02 19:31:56,234: INFO: common: Created directory at artifacts]
[2025-08-02 19:31:56,234: INFO: 886728863: ConfigurationManager initialized successfully]
[2025-08-02 19:31:56,235: INFO: common: Created directory at artifacts/model_trainer]
[2025-08-02 19:31:56,764: INFO: 97880468: Model training completed]
